# [Eval] Evaluate Trained Custom Speech Models
This sample demonstrates how to evaluate Trained Custom Speech models calling REST API. 

> ✨ ***Note*** <br>
> You can test the accuracy of your custom model by creating a test. A test requires a collection of audio files and their corresponding transcriptions. You can compare a custom model's accuracy with a speech to text base model or another custom model. After you get the test results, evaluate the word error rate (WER) compared to speech recognition results. 

## Prerequisites
Configure a Python virtual environment for 3.10 or later: 
 1. open the Command Palette (Ctrl+Shift+P).
 1. Search for Python: Create Environment.
 1. select Venv / Conda and choose where to create the new environment.
 1. Select the Python interpreter version. Create with version 3.10 or later.


## Setup the environment

In [ ]:
import azure.cognitiveservices.speech as speechsdk
import os
import json
from openai import AzureOpenAI
import requests
from dotenv import load_dotenv
from utils.common import *
import requests
import time
import json
from azure.storage.blob import BlobServiceClient, BlobClient, ContainerClient, generate_blob_sas, BlobSasPermissions
import os
import datetime
from tqdm import tqdm
import pandas as pd
from IPython.display import display, HTML

load_dotenv()

SPEECH_KEY = os.getenv("AZURE_AI_SPEECH_API_KEY")
SPEECH_REGION = os.getenv("AZURE_AI_SPEECH_REGION")
CUSTOM_SPEECH_LANG = os.getenv("CUSTOM_SPEECH_LANG")
CUSTOM_SPEECH_LOCALE = os.getenv("CUSTOM_SPEECH_LOCALE")

# Get the project and custom model IDs from the previous notebook
project_id = ""
custom_model_with_plain_id = ""
custom_model_with_acoustic_id = ""
%store -r project_id
%store -r custom_model_with_plain_id
%store -r custom_model_with_acoustic_id

try:
    project_id, custom_model_with_plain_id, custom_model_with_acoustic_id
except NameError:
    print("++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++")
    print("[ERROR] Please run the previous notebook again.")
    print("++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++")

print(project_id, custom_model_with_plain_id, custom_model_with_acoustic_id)

## 1. Test based speech models
- In order to learn how to quantitatively measure and improve the accuracy of the base speech to text model or your own custom models check this link
- https://learn.microsoft.com/en-us/azure/ai-services/speech-service/how-to-custom-speech-evaluate-data?pivots=speech-cli#create-a-test

To evaluate the word error rate (WER) of a base model in Azure AI’s Speech service, follow these steps:

Sign in to the Speech Studio:
Go to the Azure Speech Studio.
Create a Test:
Navigate to Custom speech and select your project.
Go to Test models and click on Create new test.
Select Evaluate accuracy and click Next.
Choose an audio + human-labeled transcription dataset. If you don’t have any datasets, upload them in the Speech datasets menu.
Select up to two models to evaluate, then click Next.
Enter the test name and description, then click Next.
Review the test details and click Save and close.
Get Test Results:
After the test is complete, indicated by the status set to Succeeded, you will see the results, including the WER for each tested model.
Evaluate WER:
WER is calculated as the sum of insertion, deletion, and substitution errors divided by the total number of words in the reference transcript, multiplied by 100 to get a percentage1.
For more detailed instructions, you can refer to this link - https://learn.microsoft.com/en-us/azure/ai-services/speech-service/how-to-custom-speech-evaluate-data?pivots=rest-api.


In [ ]:
# Base URL for the Speech Services REST API
base_url = f'https://{SPEECH_REGION}.api.cognitive.microsoft.com/speechtotext'

# Headers for authentication
headers = {
    'Ocp-Apim-Subscription-Key': SPEECH_KEY,
    'Content-Type': 'application/json'
}

### Check the custom speech model ids to evaluate

In [ ]:
#option1. check the model id from the train a new model (UI) in the Azure Speech Studio. 
base_model_id = "8066b5fb-0114-4837-90b6-0c245928a896"  # Vietnamese base model id

#option2. check the model id from the API call
base_model = get_latest_base_model(base_url, headers, f"locale eq '{CUSTOM_SPEECH_LOCALE}' and status eq 'Succeeded'")

# Filter the base models to find the ones that support 'Language' adaptations and have the latest lastActionDateTime
filtered_models = [model for model in base_model['values'] if 'properties' in model and 'Language' in model['properties']['features'].get('supportsAdaptationsWith', [])]
if filtered_models:
	latest_model = max(filtered_models, key=lambda x: x['createdDateTime'])
	print("Latest model supporting 'Language' adaptations:")
	print(latest_model)
else:
	print("No models found that support 'Language' adaptations.")

# Get the latest model ID from the self link for example 8066b5fb-0114-4837-90b6-0c245928a896 is the model id in 'https://swedencentral.api.cognitive.microsoft.com/speechtotext/v3.2/models/base/8066b5fb-0114-4837-90b6-0c245928a896' 
base_model_id = latest_model['self'].split('/')[-1]

In [ ]:
# check the model id from the train a new model (UI) in the Azure Speech Studio. 
# The base model ids are vary from each language 
print("Latest base model id:", base_model_id)
print("custom_model_with_plain_id: ", custom_model_with_plain_id)
print("custom_model_with_acoustic_id: ", custom_model_with_acoustic_id)

### Upload zip files to a storage account and generate content urls

In [ ]:
data_folder = "eval_dataset"
account_name = os.getenv("AZURE_STORAGE_ACCOUNT_NAME")
account_key = os.getenv("AZURE_STORAGE_ACCOUNT_KEY")
container_name = os.getenv("AZURE_STORAGE_CONTAINER_NAME")

uploaded_files, url = upload_dataset_to_storage(data_folder, container_name, account_name, account_key)

In [ ]:
uploaded_files

### Create datasets for evaluation

In [ ]:
kind="Acoustic"
description = f"[eval] Dataset for evaluation the {CUSTOM_SPEECH_LANG} base model"
dataset_id = {}

for display_name in uploaded_files:
    dataset_id[display_name] = create_dataset(base_url, headers, project_id, url[display_name], kind, display_name, description, CUSTOM_SPEECH_LOCALE)

### Test accuracy of the trained Custom Speech model creating evaluations (tests)

In [ ]:
description = f"[{CUSTOM_SPEECH_LOCALE}] Evaluation of the {CUSTOM_SPEECH_LANG} base and custom model"
evaluation_ids={}
for display_name in uploaded_files:
    evaluation_ids[display_name] = create_evaluation(base_url, headers, project_id, dataset_id[display_name], base_model_id, custom_model_with_acoustic_id, f'vi_eval_base_vs_custom_{display_name}', description, CUSTOM_SPEECH_LOCALE)

In [ ]:
# Monitor the status of the run_result
def monitor_evaluation_status(evaluation_id):
    with tqdm(total=3, desc="Running Status", unit="step") as pbar:
        status = get_evaluation_status(base_url, headers, evaluation_id)
        if status == "NotStarted":
            pbar.update(1)
        while status != "Succeeded" and status != "Failed":
            if status == "Running" and pbar.n < 2:
                pbar.update(1)
            print(f"Current Status: {status}")
            time.sleep(10)
            status = get_evaluation_status(base_url, headers, evaluation_id)
        while(pbar.n < 3):
            pbar.update(1)
        print("Evaluation Completed")

In [ ]:
for display_name in uploaded_files:
    monitor_evaluation_status(evaluation_ids[display_name])

In [ ]:
# Collect WER results for each dataset
wer_results = []
eval_title = "Evaluation Results for base model and custom model: "
for display_name in uploaded_files:
    eval_info = get_evaluation_results(base_url, headers, evaluation_ids[display_name])
    eval_title = eval_title + display_name + " "
    wer_results.append({
            'Dataset': display_name,
            'WER_base_model': eval_info['properties']['wordErrorRate1'],
            'WER_custom_model': eval_info['properties']['wordErrorRate2'],
            
    })
# Create a DataFrame to display the results
print(eval_info)
wer_df = pd.DataFrame(wer_results)
print(eval_title)
print(wer_df)

In [ ]:
md_table_scoring_result(base_url, headers, evaluation_ids, uploaded_files)